## **Preparations**

In [2]:
import time
import warnings
from typing import Tuple, List, Any

import pandas as pd
from pytabkit import (
    CatBoost_TD_Classifier, CatBoost_D_Classifier, LGBM_TD_Classifier, LGBM_D_Classifier,
    XGB_TD_Classifier, XGB_D_Classifier, RealMLP_TD_Classifier
)
from sklearn.metrics import (
    roc_auc_score, recall_score, precision_score, accuracy_score,
    confusion_matrix
)
from sklearn.model_selection import train_test_split

warnings.filterwarnings("ignore")

In [3]:
# Constants
SEED = 42

## **Load & info data**

In [5]:
train_df = pd.read_csv("data/train.csv").drop(columns=["id"])
test_df = pd.read_csv("data/test.csv").drop(columns=["id"])

submision_df = pd.read_csv("data/sample_submission.csv")

In [6]:
print(train_df.shape)
train_df.head()

(593994, 12)


,annual_income,debt_to_income_ratio,credit_score,loan_amount,interest_rate,gender,marital_status,education_level,employment_status,loan_purpose,grade_subgrade,loan_paid_back
0,29367.99,0.084,736,2528.42,13.67,Female,Single,High School,Self-employed,Other,C3,1.0
1,22108.02,0.166,636,4593.10,12.92,Male,Married,Master's,Employed,Debt consolidation,D3,0.0
2,49566.20,0.097,694,17005.15,9.76,Male,Single,High School,Employed,Debt consolidation,C5,1.0
3,46858.25,0.065,533,4682.48,16.10,Female,Single,High School,Employed,Debt consolidation,F1,1.0
4,25496.70,0.053,665,12184.43,10.21,Male,Married,High School,Employed,Other,D1,1.0


In [7]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 593994 entries, 0 to 593993
Data columns (total 12 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   annual_income         593994 non-null  float64
 1   debt_to_income_ratio  593994 non-null  float64
 2   credit_score          593994 non-null  int64  
 3   loan_amount           593994 non-null  float64
 4   interest_rate         593994 non-null  float64
 5   gender                593994 non-null  object 
 6   marital_status        593994 non-null  object 
 7   education_level       593994 non-null  object 
 8   employment_status     593994 non-null  object 
 9   loan_purpose          593994 non-null  object 
 10  grade_subgrade        593994 non-null  object 
 11  loan_paid_back        593994 non-null  float64
dtypes: float64(5), int64(1), object(6)
memory usage: 54.4+ MB


### **Preparations data for training ML models**

### Split

In [8]:
X = train_df.drop(columns=["loan_paid_back"])
y = train_df["loan_paid_back"]

X_train_val, X_test, y_train_val, y_test = train_test_split(
    X, y,
    test_size=0.2,
    stratify=y,
    random_state=SEED
)

X_train, X_val, y_train, y_val = train_test_split(
    X_train_val, y_train_val,
    test_size=0.25,
    stratify=y_train_val,
    random_state=SEED
)

In [9]:
print(f"""
Train: {X_train.shape}, {X_train.shape[0] / train_df.shape[0]:.2f};
Val: {X_val.shape}, {X_val.shape[0] / train_df.shape[0]:.2f};
Test: {X_test.shape}, {X_test.shape[0] / train_df.shape[0]:.2f};
""")


Train: (356396, 11), 0.60;
Val: (118799, 11), 0.20;
Test: (118799, 11), 0.20;



Данные разделили на три части:

Train — 356 396 строк (60%): используется для обучения модели.
Validation — 118 799 строк (20%): нужна для подбора гиперпараметров и контроля переобучения.
Test — 118 799 строк (20%): используется только для финальной оценки качества модели.

---

The data was divided into three parts:

Train — 356 396 rows (60%): used for training the model.
Validation — 118 799 rows (20%): used for hyperparameter selection and overfitting control.
Test — 118 799 rows (20%): used only for the final model quality assessment.

In [10]:
# constant
CAT_COLS = X_train.select_dtypes(include=["object"]).columns.tolist()

### Methods for calculating metrics

In [11]:
def get_classification_metrics(
    y_train: pd.Series,
    y_train_pred: pd.Series,
    y_train_proba_pred: pd.Series,
    y_test: pd.Series,
    y_test_proba_pred: pd.Series,
    y_test_pred: pd.Series
) -> pd.DataFrame:
    """
    Computes core classification metrics for both train and test datasets using
    predicted class labels and predicted probabilities, and returns them in a
    structured pandas DataFrame.
    :param y_train: True labels for the training set.
    :param y_train_pred: Predicted class labels for the training set.
    :param y_train_proba_pred: Predicted probabilities for the training set.
    :param y_test: True labels for the test set.
    :param y_test_proba_pred: Predicted probabilities for the test set.
    :param y_test_pred: Predicted class labels for the test set.
    :return: A pandas DataFrame containing metrics for train, test, and their differences.
    """
    metrics = {
        "accuracy": (accuracy_score, None),
        "precision": (precision_score, None),
        "recall": (recall_score, None),
        "roc_auc": (roc_auc_score, None)
    }
    use_proba = ["roc_auc"]

    results = {}
    for metric_name, (func, _) in metrics.items():
        if metric_name in use_proba:
            train_val = func(y_train, y_train_proba_pred)
            test_val = func(y_test, y_test_proba_pred)
        else:
            train_val = func(y_train, y_train_pred)
            test_val = func(y_test, y_test_pred)

        diff_val = train_val - test_val

        results[metric_name] = [train_val, test_val, diff_val]

    return pd.DataFrame(results, index=["train", "test", "diff"])

In [12]:
def get_confusion_matrix(
    y_true: pd.Series,
    y_pred: pd.Series,
    labels: List[int] = [0, 1]
) -> pd.DataFrame:
    """
    Builds a confusion matrix pandas DataFrame.
    :param y_true: True class labels.
    :param y_pred: Predicted class labels (binary: 0/1).
    :param labels: List of class labels in the order they should appear in the matrix.
    :return: A pandas DataFrame the confusion matrix.
    """
    cm = confusion_matrix(y_true, y_pred, labels=[0, 1])
    confusion_matrix_df = pd.DataFrame(cm, index=[f"Actual_{l}" for l in labels], columns=[f"Pred_{l}" for l in labels])

    return confusion_matrix_df

In [13]:
def get_classification_report(
    model: Any,
    X_train: pd.DataFrame,
    y_train: pd.Series,
    X_test: pd.DataFrame,
    y_test: pd.Series
) -> Tuple[pd.DataFrame, pd.DataFrame]:
    """
    Computes classification metrics for a trained machine learning model.
    :param model: Trained machine learning model.
    :param X_train: Training feature set.
    :param y_train: Training target values.
    :param X_test: Test feature set.
    :param y_test: Test target values.
    :return:
        A tuple containing:
        - A pandas DataFrame with metrics for the train and test sets,
          including their differences (useful for detecting overfitting).
        - A pandas DataFrame representing the confusion matrix for the test set.
    """
    y_train_proba_pred, y_test_proba_pred = model.predict_proba(X_train)[:, 1], model.predict_proba(X_test)[:, 1]
    y_train_pred, y_test_pred = model.predict(X_train), model.predict(X_test)

    metrics_df = get_classification_metrics(
        y_train,
        y_train_pred,
        y_train_proba_pred,
        y_test,
        y_test_proba_pred,
        y_test_pred
    )

    confusion_matrix_df = get_confusion_matrix(y_test, y_test_pred)

    return metrics_df, confusion_matrix_df

## **Create ML model**

### PyTabKit base model fit

In [14]:
params = {
    "random_state": SEED
}

models_dict = {
    "CatBoost_D_Classifier": CatBoost_D_Classifier(**params),
    "CatBoost_TD_Classifier": CatBoost_TD_Classifier(**params),
    "LGBM_D_Classifier": LGBM_D_Classifier(**params),
    "LGBM_TD_Classifier": LGBM_TD_Classifier(**params),
    "XGB_D_Classifier": XGB_D_Classifier(**params),
    "XGB_TD_Classifier": XGB_TD_Classifier(**params)
}

In [15]:
chunk_list = []

for model_name, model in models_dict.items():
    start_time = time.time()

    print(f"Run fit model: {model_name}: {model}")
    model.fit(X_train, y_train, X_val, y_val, cat_col_names=CAT_COLS)

    training_time = time.time() - start_time

    print("Calculate classification report")
    metrics_df, _ = get_classification_report(model, X_train, y_train, X_test, y_test)
    metrics_df = metrics_df.reset_index(names="part_name")
    metrics_df["model_name"] = model_name
    metrics_df["training_time"] = training_time

    print(f"Metric test: \n{metrics_df[metrics_df['part_name'] == 'test'].drop(columns=['model_name', 'part_name'])}")
    chunk_list.append(metrics_df)

Run fit model: CatBoost_D_Classifier: CatBoost_D_Classifier(random_state=42)
Calculate classification report
Metric test: 
   accuracy  precision    recall   roc_auc  training_time
1  0.904747   0.907122  0.981222  0.918399      38.069711
Run fit model: CatBoost_TD_Classifier: CatBoost_TD_Classifier(random_state=42)
Calculate classification report
Metric test: 
   accuracy  precision    recall   roc_auc  training_time
1  0.905908   0.910192  0.978788  0.921092      18.729073
Run fit model: LGBM_D_Classifier: LGBM_D_Classifier(random_state=42)
Calculate classification report
Metric test: 
   accuracy  precision    recall   roc_auc  training_time
1  0.904696   0.906934  0.981401  0.918751       3.187286
Run fit model: LGBM_TD_Classifier: LGBM_TD_Classifier(random_state=42)
Calculate classification report
Metric test: 
   accuracy  precision    recall   roc_auc  training_time
1  0.905428   0.909467  0.979072  0.920092      20.239098
Run fit model: XGB_D_Classifier: XGB_D_Classifier(random

In [16]:
benchmark_df = pd.concat(chunk_list, ignore_index=True)

In [17]:
benchmark_df.groupby("model_name")["training_time"].max().sort_values()

model_name
XGB_D_Classifier           2.160020
LGBM_D_Classifier          3.187286
XGB_TD_Classifier          5.962755
CatBoost_TD_Classifier    18.729073
LGBM_TD_Classifier        20.239098
CatBoost_D_Classifier     38.069711
Name: training_time, dtype: float64

1. Лучший результат показал CatBoost_TD_Classifier (0.9211). Следом с минимальным отрывом идет LGBM_TD_Classifier (0.9201).
2. Разница между моделями очень мала (менее 0.4%).
3. Хотя CatBoost и LGBM в версиях TD дают лучший ROC_AUC, они значительно медленнее. XGB_D и LGBM_D — самые быстрые (2-3 сек), при этом их ROC_AUC (~0.918) лишь незначительно уступает лидерам. Если критично время обучения/инференса, стоит выбрать их.

---

1. The best result was shown by CatBoost_TD_Classifier (0.9211). LGBM_TD_Classifier (0.9201) comes next with a minimal margin.
2. The difference between the models is very small (less than 0.4%).
3. Although CatBoost and LGBM in TD versions give the best ROC_AUC, they are significantly slower. XGB_D and LGBM_D are the fastest (2-3 sec), while their ROC_AUC (~0.918) is only slightly inferior to the leaders. If training/inference time is critical, you should choose them.

### Ensemble boosting (n_fold=5)

In [25]:
params = {
    "n_cv": 5,
    "random_state": SEED
}

models_dict = {
    "CatBoost_D_Classifier_cv5": CatBoost_D_Classifier(**params),
    "CatBoost_TD_Classifier_cv5": CatBoost_TD_Classifier(**params),
    "LGBM_D_Classifier_cv5": LGBM_D_Classifier(**params),
    "LGBM_TD_Classifier_cv5": LGBM_TD_Classifier(**params),
    "XGB_D_Classifier_cv5": XGB_D_Classifier(**params),
    "XGB_TD_Classifier_cv5": XGB_TD_Classifier(**params)
}

In [26]:
chunk_list = []

for model_name, model in models_dict.items():
    start_time = time.time()

    print(f"Run fit model: {model_name}: {model}")
    model.fit(X_train, y_train, cat_col_names=CAT_COLS)

    training_time = time.time() - start_time

    print("Calculate classification report")
    metrics_df, _ = get_classification_report(model, X_train, y_train, X_test, y_test)
    metrics_df = metrics_df.reset_index(names="part_name")
    metrics_df["model_name"] = model_name
    metrics_df["training_time"] = training_time

    print(f"Metric test: \n{metrics_df[metrics_df['part_name'] == 'test'].drop(columns=['model_name', 'part_name'])}")
    chunk_list.append(metrics_df)

Run fit model: CatBoost_D_Classifier_cv5: CatBoost_D_Classifier(n_cv=5, random_state=42)
Calculate classification report
Metric test: 
   accuracy  precision    recall  roc_auc  training_time
1  0.905033   0.907105  0.981644  0.91835     161.678337
Run fit model: CatBoost_TD_Classifier_cv5: CatBoost_TD_Classifier(n_cv=5, random_state=42)
Calculate classification report
Metric test: 
   accuracy  precision    recall  roc_auc  training_time
1  0.905984   0.909718  0.979515  0.92099      74.501748
Run fit model: LGBM_D_Classifier_cv5: LGBM_D_Classifier(n_cv=5, random_state=42)
Calculate classification report
Metric test: 
   accuracy  precision    recall   roc_auc  training_time
1  0.904856   0.906936  0.981623  0.918891       9.623572
Run fit model: LGBM_TD_Classifier_cv5: LGBM_TD_Classifier(n_cv=5, random_state=42)
Calculate classification report
Metric test: 
   accuracy  precision    recall   roc_auc  training_time
1  0.905479   0.908832  0.979979  0.920378      70.404154
Run fit mode

In [27]:
ensemble_df = pd.concat(chunk_list, ignore_index=True)

In [28]:
ensemble_df.groupby("model_name")["training_time"].max().sort_values()

model_name
XGB_D_Classifier_cv5            4.795875
LGBM_D_Classifier_cv5           9.623572
XGB_TD_Classifier_cv5          20.976008
LGBM_TD_Classifier_cv5         70.404154
CatBoost_TD_Classifier_cv5     74.501748
CatBoost_D_Classifier_cv5     161.678337
Name: training_time, dtype: float64

In [30]:
benchmark_df = pd.concat([benchmark_df, ensemble_df], ignore_index=True)

1. Первое место сохраняет CatBoost_TD_cv5 (0.9210)
2. По сравнению с одиночными моделями из прошлого запуска, прирост качества от ансамблирования минимален или отсутствует (разница в 3-4 знаке).
3. XGB_D_cv5 - ансамблирование дало заметный прирост (с 0.9184 до 0.9196), при этом он обучился всего за 4.8 сек. Это лучший вариант по соотношению скорость/качество.
4. CatBoost и LGBM в версиях TD дают высокое качество, но ценой значительного времени (70–161 сек), что в 15–30 раз медленнее XGBoost.

---

1. The first place is retained by CatBoost_TD_cv5 (0.9210)
2. Compared to single models from the previous run, the quality gain from ensemble is minimal or absent (difference in 3-4 sign).
3. XGB_D_cv5 - ensemble gave a noticeable gain (from 0.9184 to 0.9196), while it was trained in just 4.8 sec. This is the best option in terms of speed/quality.
4. CatBoost and LGBM in the TD versions give high quality, but at the cost of significant time (70–161 sec), which is 15–30 times slower than XGBoost.

### RealMLP

In [31]:
params = {
    "n_epochs": 64,
    "val_metric_name": "1-auc_ovr",
    "random_state": SEED,
    "verbosity": 200
}

In [32]:
model = RealMLP_TD_Classifier(**params)

In [33]:
start_time = time.time()
model.fit(X_train, y_train, X_val, y_val, cat_col_names=CAT_COLS)
training_time = time.time() - start_time

Columns classified as continuous: ['annual_income', 'debt_to_income_ratio', 'credit_score', 'loan_amount', 'interest_rate']
Columns classified as categorical: ['gender', 'marital_status', 'education_level', 'employment_status', 'loan_purpose', 'grade_subgrade']


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]


moving static model to (device(type='cuda', index=0),) {}
Epoch 1/64: val 1-auc_ovr = 0.088935
Epoch 2/64: val 1-auc_ovr = 0.093587
Epoch 3/64: val 1-auc_ovr = 0.088185
Epoch 4/64: val 1-auc_ovr = 0.087703
Epoch 5/64: val 1-auc_ovr = 0.087403
Epoch 6/64: val 1-auc_ovr = 0.088412
Epoch 7/64: val 1-auc_ovr = 0.088605
Epoch 8/64: val 1-auc_ovr = 0.088432
Epoch 9/64: val 1-auc_ovr = 0.088245
Epoch 10/64: val 1-auc_ovr = 0.087868
Epoch 11/64: val 1-auc_ovr = 0.087452
Epoch 12/64: val 1-auc_ovr = 0.087439
Epoch 13/64: val 1-auc_ovr = 0.087026
Epoch 14/64: val 1-auc_ovr = 0.087107
Epoch 15/64: val 1-auc_ovr = 0.088115
Epoch 16/64: val 1-auc_ovr = 0.088469
Epoch 17/64: val 1-auc_ovr = 0.088113
Epoch 18/64: val 1-auc_ovr = 0.089052
Epoch 19/64: val 1-auc_ovr = 0.088512
Epoch 20/64: val 1-auc_ovr = 0.089188
Epoch 21/64: val 1-auc_ovr = 0.090600
Epoch 22/64: val 1-auc_ovr = 0.087572
Epoch 23/64: val 1-auc_ovr = 0.091813
Epoch 24/64: val 1-auc_ovr = 0.087449
Epoch 25/64: val 1-auc_ovr = 0.087930
E

`Trainer.fit` stopped: `max_epochs=64` reached.


In [34]:
metrics_df, _ = get_classification_report(model, X_train, y_train, X_test, y_test)

metrics_df = metrics_df.reset_index(names="part_name")
metrics_df["model_name"] = "RealMLP_TD_Classifier"
metrics_df["training_time"] = training_time

ERROR! Session/line number was not unique in database. History logging moved to new session 31


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]


moving static model to ('cuda:0',) {}
moving static model to (device(type='cuda', index=0),) {}


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]


moving static model to ('cuda:0',) {}
moving static model to (device(type='cuda', index=0),) {}


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]


moving static model to ('cuda:0',) {}
moving static model to (device(type='cuda', index=0),) {}


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]


moving static model to ('cuda:0',) {}
moving static model to (device(type='cuda', index=0),) {}


In [35]:
metrics_df

,part_name,accuracy,precision,recall,roc_auc,model_name,training_time
0,train,0.902387,0.903241,0.983119,0.914307,RealMLP_TD_Classifier,706.69461
1,test,0.901641,0.902794,0.982676,0.912355,RealMLP_TD_Classifier,706.69461
2,diff,0.000746,0.000447,0.000443,0.001952,RealMLP_TD_Classifier,706.69461


In [36]:
benchmark_df = pd.concat([benchmark_df, metrics_df], ignore_index=True)

1. Это худший результат среди всех протестированных моделей (0.9124).
2. Модель RealMLP в данном эксперименте неконкурентоспособна. Она самая медленная и самая слабая по метрике ROC_AUC.
3. Модель в 10 раз медленнее, чем тяжелые ансамбли CatBoost (75 сек) и в 350 раз медленнее, чем XGBoost (2 сек).

---

1. This is the worst result among all the tested models (0.9124).
2. The RealMLP model is not competitive in this experiment. It is the slowest and weakest model in terms of the ROC_AUC metric.
3. The model is 10 times slower than the heavy CatBoost ensembles (75 seconds) and 350 times slower than XGBoost (2 seconds).

### Ensemble RealMLP

In [37]:
params = {
    "n_cv": 5,
    "n_epochs": 64,
    "val_metric_name": "1-auc_ovr",
    "random_state": SEED,
    "verbosity": 200
}

In [38]:
model = RealMLP_TD_Classifier(**params)

In [39]:
start_time = time.time()
model.fit(X_train, y_train, X_val, y_val, cat_col_names=CAT_COLS)
training_time = time.time() - start_time

Columns classified as continuous: ['annual_income', 'debt_to_income_ratio', 'credit_score', 'loan_amount', 'interest_rate']
Columns classified as categorical: ['gender', 'marital_status', 'education_level', 'employment_status', 'loan_purpose', 'grade_subgrade']


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]


moving static model to (device(type='cuda', index=0),) {}
Epoch 1/64: val 1-auc_ovr = 0.091960
Epoch 2/64: val 1-auc_ovr = 0.089083
Epoch 3/64: val 1-auc_ovr = 0.088061
Epoch 4/64: val 1-auc_ovr = 0.087417
Epoch 5/64: val 1-auc_ovr = 0.088190
Epoch 6/64: val 1-auc_ovr = 0.088286
Epoch 7/64: val 1-auc_ovr = 0.089226
Epoch 8/64: val 1-auc_ovr = 0.088885
Epoch 9/64: val 1-auc_ovr = 0.088533
Epoch 10/64: val 1-auc_ovr = 0.087997
Epoch 11/64: val 1-auc_ovr = 0.087605
Epoch 12/64: val 1-auc_ovr = 0.087150
Epoch 13/64: val 1-auc_ovr = 0.087055
Epoch 14/64: val 1-auc_ovr = 0.087375
Epoch 15/64: val 1-auc_ovr = 0.087541
Epoch 16/64: val 1-auc_ovr = 0.088025
Epoch 17/64: val 1-auc_ovr = 0.088317
Epoch 18/64: val 1-auc_ovr = 0.089059
Epoch 19/64: val 1-auc_ovr = 0.089371
Epoch 20/64: val 1-auc_ovr = 0.090023
Epoch 21/64: val 1-auc_ovr = 0.089292
Epoch 22/64: val 1-auc_ovr = 0.089598
Epoch 23/64: val 1-auc_ovr = 0.088338
Epoch 24/64: val 1-auc_ovr = 0.088843
Epoch 25/64: val 1-auc_ovr = 0.087845
E

`Trainer.fit` stopped: `max_epochs=64` reached.


In [40]:
metrics_df, _ = get_classification_report(model, X_train, y_train, X_test, y_test)

metrics_df = metrics_df.reset_index(names="part_name")
metrics_df["model_name"] = "RealMLP_TD_Classifier_cv5"
metrics_df["training_time"] = training_time

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]


moving static model to ('cuda:0',) {}
moving static model to (device(type='cuda', index=0),) {}


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]


moving static model to ('cuda:0',) {}
moving static model to (device(type='cuda', index=0),) {}


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]


moving static model to ('cuda:0',) {}
moving static model to (device(type='cuda', index=0),) {}


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]


moving static model to ('cuda:0',) {}
moving static model to (device(type='cuda', index=0),) {}


In [41]:
metrics_df

,part_name,accuracy,precision,recall,roc_auc,model_name,training_time
0,train,0.902389,0.904120,0.981939,0.914373,RealMLP_TD_Classifier_cv5,749.52441
1,test,0.902087,0.903907,0.981802,0.913053,RealMLP_TD_Classifier_cv5,749.52441
2,diff,0.000303,0.000214,0.000137,0.001320,RealMLP_TD_Classifier_cv5,749.52441


In [42]:
benchmark_df = pd.concat([benchmark_df, metrics_df], ignore_index=True)

1. По сравнению с одиночной RealMLP (0.9124), версия с кросс-валидацией показала минимальный прирост качества (+0.0007 ROC_AUC). Ансамблирование нейросетей не дало качественного скачка.

---

1. Compared to a single RealMLP (0.9124), the cross-validation version showed a minimal increase in quality (+0.0007 ROC_AUC). Neural network ensemble did not provide a significant improvement.